# Лабораторна робота 1
### Виконав ФБ-21 Захожий Микола
<b> Мета роботи:</b> ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції

1. Для кожної із адміністративних одиниць України завантажити текстові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [133]:
import os
import pandas as pd
import urllib.request
from datetime import datetime

In [77]:

path = '/home/kunopohui/Downloads/csv'
def get_data(province_id):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)

    now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    # path = '/home/kunopohui/Downloads/csv'
    filename = f'vhi_id_{province_id}_{now}.csv'
    filepath = f'{path}/{filename}'
    with open(filepath, 'wb') as out:
        out.write(vhi_url.read())
    print(f"VHI was downloaded in {filepath}")

    return

for i in range(1,28):
    get_data(i)

VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_1_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_2_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_3_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_4_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_5_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_6_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_7_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_8_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_9_2024-04-05_12-32.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_10_2024-04-05_12-33.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_11_2024-04-05_12-33.csv
VHI was downloaded in /home/kunopohui/Downloads/csv/vhi_id_12_2024-04-05_12-33.csv
VHI was downl

In [80]:
state_id_name = {
        1: 'Vinnytsia', 13: 'Mykolaiv',
        2: 'Volyn', 14: 'Odesa',
        3: 'Dnipropetrovska', 15: 'Poltavska',
        4: 'Donetsk', 16: 'Rivne',
        5: 'Zhytomyr', 17: 'Sumy',
        6: 'Zakarpattia', 18: 'Ternopil',
        7: 'Zaporizhzhya', 19: 'Kharkiv',
        8: 'Ivano-Frankivsk', 20: 'Kherson',
        9: 'Kyiv', 21: 'Khmelnytsky',
        10: 'Kirovohrad', 22: 'Cherkasy',
        11: 'Luhansk', 23: 'Chernivtsi',
        12: 'Lviv', 24: 'Chernihiv',
        25: 'Republic of Crimea'
}



2. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [98]:
def read_and_create_data_frame(path):
    
    out_df = pd.DataFrame()

    csv_files = os.listdir('/home/kunopohui/Downloads/csv')

    frames = []

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']


    for file in range(len(csv_files)):
        file_path = os.path.join(path, csv_files[file])
        df = pd.read_csv(file_path , header = 1, names = headers)
        
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['Year'] = df['Year'].str.replace('<tt><pre>', '')
        df = df[~df['Year'].str.contains('</pre></tt>')]
        df['area'] = file + 1
        df.drop('empty', axis=1, inplace=True)
        frames.append(df)
    
    out_df = pd.concat(frames).drop_duplicates().reset_index(drop=True)
    out_df.drop([22, 12])
    out_df = out_df.loc[(out_df.area != 12) & (out_df.area != 20)]  
    return out_df
 
df = read_and_create_data_frame('/home/kunopohui/Downloads/csv')
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.053  260.31  45.01  39.46  42.23     1
1      1982   2.0  0.054  262.29  46.83  31.75  39.29     1
2      1982   3.0  0.055  263.82  48.13  27.24  37.68     1
3      1982   4.0  0.053  265.33  46.09  23.91  35.00     1
4      1982   5.0  0.050  265.66  41.46  26.65  34.06     1
...     ...   ...    ...     ...    ...    ...    ...   ...
57964  2024   9.0  0.159  275.16  72.78  20.87  46.83    27
57965  2024  10.0  0.172  277.53  72.73  19.51  46.12    27
57966  2024  11.0  0.180  278.93  68.66  21.11  44.89    27
57967  2024  12.0  0.183  279.90  62.37  28.02  45.19    27
57968  2024  13.0  0.190  281.04  57.21  34.39  45.80    27

[53675 rows x 8 columns]


3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на потрібні.

In [153]:
def NewIDS(df):
    province_indexes = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    df.replace({'area':province_indexes}, inplace = True)
    df['area'] = df['area'].replace(state_id_name)
    return df

print(NewIDS(df))





       Year  Week    SMN     SMT    VCI    TCI    VHI      area
0      1982   1.0  0.053  260.31  45.01  39.46  42.23  Cherkasy
1      1982   2.0  0.054  262.29  46.83  31.75  39.29  Cherkasy
2      1982   3.0  0.055  263.82  48.13  27.24  37.68  Cherkasy
3      1982   4.0  0.053  265.33  46.09  23.91  35.00  Cherkasy
4      1982   5.0  0.050  265.66  41.46  26.65  34.06  Cherkasy
...     ...   ...    ...     ...    ...    ...    ...       ...
57964  2024   9.0  0.159  275.16  72.78  20.87  46.83  Zhytomyr
57965  2024  10.0  0.172  277.53  72.73  19.51  46.12  Zhytomyr
57966  2024  11.0  0.180  278.93  68.66  21.11  44.89  Zhytomyr
57967  2024  12.0  0.183  279.90  62.37  28.02  45.19  Zhytomyr
57968  2024  13.0  0.190  281.04  57.21  34.39  45.80  Zhytomyr

[53675 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду:
Ряд VHI для області за рік, пошук екстремумів (min та max);

In [135]:
def find_max_vhi(df, area, year):

  filtered_df = df[ (df['area'] == area) & (df['Year'] == year) ]
  if filtered_df.empty:
    return None  
  return filtered_df['VHI'].max()


def find_min_vhi(df, area, year):

  filtered_df = df[ (df['area'] == area) & (df['Year'] == year) ]
  if filtered_df.empty:
    return None  
  return filtered_df['VHI'].min()

def analyze_vhi(df, year):

  for area in df['area'].unique():
    max_vhi = find_max_vhi(df.copy(), area, year) 
    min_vhi = find_min_vhi(df.copy(), area, year) 
    print(f'{area}, max: {max_vhi}, min:{min_vhi} ')

analyze_vhi(df, '2022')


Cherkasy, max: 72.32, min:36.12 
Chernihiv, max: 78.8, min:37.84 
Chernivtsi, max: 57.39, min:37.64 
Republic of Crimea, max: 60.93, min:34.83 
Dnipropetrovska, max: 61.06, min:39.52 
Donetsk, max: 61.99, min:38.31 
Ivano-Frankivsk, max: 56.41, min:37.34 
Kharkiv, max: 67.19, min:38.47 
Kherson, max: 54.81, min:35.7 
Khmelnytsky, max: 62.81, min:36.56 
Kyiv, max: 65.14, min:30.32 
Kirovohrad, max: 73.98, min:36.45 
Luhansk, max: 63.71, min:41.93 
Lviv, max: 56.28, min:37.65 
Mykolaiv, max: 58.87, min:38.86 
Odesa, max: 53.07, min:32.11 
Poltavska, max: 80.55, min:37.74 
Rivne, max: 64.19, min:33.37 
Sumy, max: 81.96, min:40.3 
Ternopil, max: 61.82, min:36.58 
Zakarpattia, max: 53.83, min:38.47 
Vinnytsia, max: 56.9, min:32.28 
Volyn, max: 61.15, min:38.38 
Zaporizhzhya, max: 61.73, min:36.74 
Zhytomyr, max: 63.81, min:30.36 


Ряд VHI за вказаний діапазон років для вказаних областей;
виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);
Аналогічно для помірних посух

In [188]:
def drought_years2(df, state_percent, drought_percent_less, drought_percent_more):

  drought = df[(df.VHI >= drought_percent_less) & (df.VHI <= drought_percent_more)]
  state_number = (state_percent/100)*df['area'].nunique()
  yearslist = []
  for year in df['Year'].unique():
    drought_per_year = drought[drought['Year'] == year ]  
    drought_area_number = len(drought_per_year['area'].unique())
    # print(drought_area_number)
    if drought_area_number >=state_number:
       yearslist.append(year)
  return yearslist

print(drought_years2(df, 20, 15, 35))
print(drought_years2(df, 20, 0, 15))

['1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
['2007']
